In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from monty.serialization import dumpfn, loadfn
from collections import defaultdict
from ase.filters import FrechetCellFilter
from ase.optimize import LBFGS
from pymatgen.io.ase import AseAtomsAdaptor
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from doped.generation import DefectsGenerator
from shakenbreak.input import Distortions
from pymatgen.core import Element

/home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL = "uma"

In [4]:
uma_ff = pretrained_mlip.get_predict_unit("uma-m-1p1", device="cuda")
uma_calc = FAIRChemCalculator(uma_ff, task_name="omat")

MODELS = {
    "uma":uma_calc
}

In [5]:
def relax_structure(structure):
    atoms = structure.to_ase_atoms()
    atoms.calc = MODELS[MODEL]

    opt = LBFGS(FrechetCellFilter(atoms)) # Delete this FrechetCellFilter when doing the defect structures
    opt.run(0.01, 1000)

    relaxed_structure = AseAtomsAdaptor.get_structure(atoms)
    energy = atoms.get_potential_energy()

    return relaxed_structure, energy 

In [6]:
def relax_defect(structure): # takes in pymatgen structure
    atoms = structure.to_ase_atoms()
    atoms.calc = MODELS[MODEL]

    opt = LBFGS(atoms) # Delete this FrechetCellFilter when doing the defect structures
    opt.run(0.01, 1000)

    relaxed_structure = AseAtomsAdaptor.get_structure(atoms)
    energy = atoms.get_potential_energy()

    return relaxed_structure, energy 

### Relaxation of All Primitive Cells

In [7]:
matching_pairs = loadfn('../../data/matching_extended_screening.json')

In [8]:
for name, val in matching_pairs.items():
    structure, energy = relax_structure(val['structure'])
    matching_pairs[name]['primitive relaxation'] = {
        'formula': val['formula'],
        'energy': energy,
        'structure': structure
    }

       Step     Time          Energy          fmax
LBFGS:    0 16:31:49      -35.136176        0.418707
LBFGS:    1 16:31:50      -35.143677        0.370641
LBFGS:    2 16:31:50      -35.164570        0.099608
LBFGS:    3 16:31:51      -35.165521        0.092845
LBFGS:    4 16:31:51      -35.169802        0.144372
LBFGS:    5 16:31:52      -35.171763        0.139611
LBFGS:    6 16:31:52      -35.173039        0.094901
LBFGS:    7 16:31:52      -35.173705        0.090841
LBFGS:    8 16:31:53      -35.174486        0.122176
LBFGS:    9 16:31:53      -35.175406        0.141393
LBFGS:   10 16:31:53      -35.176789        0.148430
LBFGS:   11 16:31:54      -35.178542        0.136817
LBFGS:   12 16:31:54      -35.180455        0.129848
LBFGS:   13 16:31:55      -35.182440        0.170959
LBFGS:   14 16:31:55      -35.184985        0.179349
LBFGS:   15 16:31:55      -35.188716        0.241360
LBFGS:   16 16:31:56      -35.193341        0.244484
LBFGS:   17 16:31:56      -35.196823        0.12

_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 2.273502311306314e-13


LBFGS:    7 16:35:09     -231.344291        0.410857


_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 2.2410746579354034e-13


LBFGS:    8 16:35:09     -231.377429        0.106871


...

LBFGS:   61 16:42:20     -246.619465        0.017110


_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 3.0225195427965666e-13


LBFGS:   62 16:42:20     -246.619564        0.014718


_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 3.051630489003261e-13


LBFGS:   63 16:42:21     -246.619635        0.009764
       Step     Time          Energy          fmax
LBFGS:    0 16:42:21     -221.124904        0.637964
LBFGS:    1 16:42:22     -221.194335        0.391945
LBFGS:    2 16:42:22     -221.215238        0.349599
LBFGS:    3 16:42:23     -221.261656        0.252034
LBFGS:    4 16:42:23     -221.275496        0.237226
LBFGS:    5 16:42:24     -221.289271        0.227244
LBFGS:    6 16:42:24     -221.307634        0.236589
LBFGS:    7 16:42:25     -221.330881        0.254897
LBFGS:    8 16:42:25     -221.348548        0.182429
LBFGS:    9 16:42:26     -221.358050        0.110678
LBFGS:   10 16:42:26     -221.363432        0.125774
LBFGS:   11 16:42:27     -221.368728        0.122693
LBFGS:   12 16:42:27     -221.376842        0.143391
LBFGS:   13 16:42:28     -221.388656        0.203806
LBFGS:   14 16:42:28     -221.403173        0.216799
LBFGS:   15 16:42:29     -221.415934        0.190125
LBFGS:   16 16:42:29     -221.424579        0.13

In [9]:
dumpfn(matching_pairs, f'../../data/{MODEL}/relaxed_tm_prim.json')

### Defect Creation and Relaxation

In [17]:
supercell_size = 10.1

In [18]:
def create_defect(structure, dopant, metal):
    oxidation_states = structure.composition.oxi_state_guesses()[0]
    defect_gen = DefectsGenerator(structure, extrinsic=dopant, interstitial_gen_kwargs=False, supercell_gen_kwargs={'min_image_distance':supercell_size} ) # set large super cell 25 Angstroms 'supercell_gen_kwargs={'min_image_distance':15}'
    valid_keys = []
    for key in defect_gen.keys():
        if  f"{dopant}_{metal}" in key and key[-1] == "0":
            valid_keys.append(key)
    Dist = Distortions(
        defect_entries = {key: defect_gen[key] for key in valid_keys}, # NO Cr_Al_0 for Al2(SO4)3
        oxidation_states={dopant:oxidation_states[metal]}# oxidation states predicted are good for now
    ) #try setting dopant same as metal
    distorted_defects_dict, Dist.distortion_metadata = Dist.apply_distortions()
    unperturbed_structures = {}
    distortions = {}
    for key in valid_keys:
        unperturbed_structures[key] = distorted_defects_dict[key[:-2]]['charges'][0]['structures']['Unperturbed']
        distortions[key] = distorted_defects_dict[key[:-2]]['charges'][0]['structures']['distortions']

    return dict(unperturbed_structures), dict(distortions), defect_gen.bulk_supercell

In [19]:
def get_energy(unperturbed_structures, distortions, supercell):
    data = defaultdict(dict)
    supercell_struct, supercell_e = relax_defect(supercell)
    data['supercell'] = {
        'structure': supercell_struct,
        'energy': supercell_e
    }
    
    for site, unperturbed_structure in unperturbed_structures.items():
        data[site] = defaultdict(dict)
        data[site]['unperturbed'] = defaultdict(dict)
        unperturbed_structure1, unp_energy = relax_defect(unperturbed_structure)
        data[site]['unperturbed'] = {
        'structure':unperturbed_structure1,
        'energy': unp_energy,
        'distortion_amount':'N/A'
        }
        data[site]['distortions'] = defaultdict(dict)
        for key, structure in distortions[site].items():
            structure1, energy = relax_defect(structure)
            if key != 'Rattled':
                distortion_amount = float(key.split("_")[-1].replace("%", "").replace("-", "-"))/100
            else:
                distortion_amount = 0
            data[site]['distortions'][key] = {
                'structure': structure1,
                'energy': energy,
                'distortion_amount': distortion_amount,
                'delta_e': energy - unp_energy
            }
    
    return dict(data)

In [20]:
prim = loadfn(f'../../data/{MODEL}/relaxed_tm_prim.json')

In [21]:
for key, val in prim.items():# change to if key in faulty
        unperturbed_structures, distortions, supercell = create_defect(val['structure'], val['dopant'], val['metal'])
        mini_data = get_energy(unperturbed_structures, distortions, supercell) # add in host supercell relaxation
        prim[key]['defect-bulk-supercells'] = mini_data

Generating DefectEntry objects: 100.0%|████████████████████████████████████████████████████████████| [00:25,   3.90it/s]


Vacancies    Guessed Charges    Conv. Cell Coords    Wyckoff
-----------  -----------------  -------------------  ---------
v_Al         [+1,0,-1,-2,-3]    [0.000,0.500,0.104]  2b
v_Cl         [+1,0,-1]          [0.000,0.000,0.686]  2a
v_O          [+2,+1,0,-1]       [0.000,0.000,0.049]  2a

Substitutions    Guessed Charges           Conv. Cell Coords    Wyckoff
---------------  ------------------------  -------------------  ---------
V_Al             [+2,+1,0,-1]              [0.000,0.500,0.104]  2b
Al_Cl            [+4,+3,+2,+1,0]           [0.000,0.000,0.686]  2a
Al_O             [+5,+4,+3,+2,+1,0]        [0.000,0.000,0.049]  2a
Cl_Al            [0,-1,-2,-3,-4]           [0.000,0.500,0.104]  2b
V_Cl             [+6,+5,+4,+3,+2,+1,0]     [0.000,0.000,0.686]  2a
Cl_O             [+1,0]                    [0.000,0.000,0.049]  2a
O_Al             [0,-1,-2,-3,-4,-5]        [0.000,0.500,0.104]  2b
O_Cl             [0,-1]                    [0.000,0.000,0.686]  2a
V_O              [+7,+6,+

Generating distorted defect structures...100.0%|███████████████████████████████████████████████████| [00:00,   6.10it/s]



Defect: V_Al
Number of missing electrons in neutral state: 0

Defect V_Al in charge state: 0. Number of distorted neighbours: 0
       Step     Time          Energy          fmax
LBFGS:    0 16:46:57     -386.497972        0.180237
LBFGS:    1 16:46:58     -386.520746        0.152764
LBFGS:    2 16:46:58     -386.587473        0.074327
LBFGS:    3 16:46:59     -386.589899        0.051042
LBFGS:    4 16:46:59     -386.592356        0.032627
LBFGS:    5 16:47:00     -386.592875        0.025351
LBFGS:    6 16:47:01     -386.593378        0.017444
LBFGS:    7 16:47:01     -386.593615        0.009110
       Step     Time          Energy          fmax
LBFGS:    0 16:47:03     -387.388998        2.795060
LBFGS:    1 16:47:06     -387.720602        1.443606
LBFGS:    2 16:47:08     -387.929556        0.833652
LBFGS:    3 16:47:10     -388.004705        0.639212
LBFGS:    4 16:47:13     -388.057371        0.489937
LBFGS:    5 16:47:15     -388.078443        0.324870
LBFGS:    6 16:47:18     -3

Generating DefectEntry objects: 100.0%|████████████████████████████████████████████████████████████| [00:25,   3.94it/s]


...

Vacancies                 Guessed Charges    Conv. Cell Coords    Wyckoff
------------------------  -----------------  -------------------  ---------
v_Mg_C1_O2.01O2.05        [+1,0,-1,-2]       [0.749,0.258,0.025]  4a
v_Mg_C1_O2.01O2.07        [+1,0,-1,-2]       [0.530,0.016,0.250]  4a
v_Mg_C1_O2.02             [+1,0,-1,-2]       [0.031,0.516,0.247]  4a
v_B_C1_O1.37              [+1,0,-1,-2,-3]    [0.249,0.045,0.399]  4a
v_B_C1_O1.44O1.55         [+1,0,-1,-2,-3]    [0.250,0.406,0.427]  4a
v_B_C1_O1.44O1.56B2.45    [+1,0,-1,-2,-3]    [0.097,0.247,0.080]  4a
v_B_C1_O1.44O1.56B2.46    [+1,0,-1,-2,-3]    [0.403,0.247,0.080]  4a
v_B_C1_O1.45O1.50B2.45    [+1,0,-1,-2,-3]    [0.753,0.756,0.752]  4a
v_B_C1_O1.45O1.50B2.46    [+1,0,-1,-2,-3]    [0.503,0.002,0.000]  4a
v_B_C1_O1.46              [+1,0,-1,-2,-3]    [0.002,0.998,0.499]  4a
v_Cl                      [+1,0,-1]          [0.250,0.725,0.238]  4a
v_O_C1_B1.37B1.50Mg2.05   [+2,+1,0,-1]       [0.671,0.457,0.475]  4a
v_O_C1_B1.37B1.50Mg2.0

Generating distorted defect structures...33.3%|█████████████████▎                                  | [00:00,   7.67it/s]


Defect: Ni_Mg_C1_O2.01O2.05
Number of missing electrons in neutral state: 0

Defect Ni_Mg_C1_O2.01O2.05 in charge state: 0. Number of distorted neighbours: 0

Defect: Ni_Mg_C1_O2.01O2.07
Number of missing electrons in neutral state: 0

Defect Ni_Mg_C1_O2.01O2.07 in charge state: 0. Number of distorted neighbours: 0


Generating distorted defect structures...100.0%|███████████████████████████████████████████████████| [00:00,   8.47it/s]



Defect: Ni_Mg_C1_O2.02
Number of missing electrons in neutral state: 0

Defect Ni_Mg_C1_O2.02 in charge state: 0. Number of distorted neighbours: 0
       Step     Time          Energy          fmax
LBFGS:    0 22:44:00    -1423.997266        0.291842
LBFGS:    1 22:44:02    -1424.066022        0.143335
LBFGS:    2 22:44:05    -1424.105481        0.079771
LBFGS:    3 22:44:07    -1424.112684        0.071739
LBFGS:    4 22:44:09    -1424.119916        0.055158
LBFGS:    5 22:44:11    -1424.123853        0.064431
LBFGS:    6 22:44:13    -1424.129255        0.064176
LBFGS:    7 22:44:15    -1424.132551        0.033916
LBFGS:    8 22:44:18    -1424.134168        0.025751
LBFGS:    9 22:44:20    -1424.134900        0.025279
LBFGS:   10 22:44:22    -1424.135724        0.028599
LBFGS:   11 22:44:24    -1424.136487        0.020583
LBFGS:   12 22:44:27    -1424.137006        0.015464
LBFGS:   13 22:44:29    -1424.137311        0.010121
LBFGS:   14 22:44:31    -1424.137494        0.012058
LBFGS

In [23]:
for key,val in prim.items(): # goes through mp id's
    min_energy = [float('inf'), '']
    for key1, val1 in val['defect-bulk-supercells'].items(): # goes through supercell and sites
        if key1 != 'supercell' and key1 != 'formula':
            for key2, val2 in val1.items(): 
                if key2 == 'unperturbed':
                    if val2['energy'] < min_energy[0]:
                        min_energy[0] = val2['energy']
                        min_energy[1] = [key1, key2]
                else:
                    for key3, val3 in val2.items():
                        if val3['energy'] < min_energy[0]:
                            min_energy[0] = val3['energy']
                            min_energy[1] = [key1, key2, key3]
    prim[key]['minimum energy site'] = {
        'energy': min_energy[0],
        'information': min_energy[1]
    }

In [24]:
dumpfn(prim, f'../../data/{MODEL}/relaxed_supercell_tm.json')